# Welcome to Cristian Fontana's Final Capstone Assingment

### Buenos Aires geodata Market Analysis by people income.

## 1) Introduction/Business Problem

Clearly define a problem or an idea of your choice, where you would need to leverage the Foursquare location data to solve or execute. Remember that data science problems always target an audience and are meant to help a group of stakeholders solve a problem, so make sure that you explicitly describe your audience and why they would care about your problem.

When you want to open a new restaurant in Buenos Aires, one of the keys of the sucess is the location, that you need to think about is where you should open it.
We will help stakeholder deciding where to open the restaurant based on the average income, population of the communes and competition on the zone.

## 2) Data

Describe the data that you will be using to solve the problem or execute your idea. Remember that you will need to use the Foursquare location data to solve the problem or execute your idea. You can absolutely use other datasets in combination with the Foursquare location data. So make sure that you provide adequate explanation and discussion, with examples, of the data that you will be using, even if it is only Foursquare location data.

### 2.1) Data Goal

To provide the stakeholders the necessary information I'll be combining Buenos Aires's 2018 Census that contains Average income per commune, from wikipedia I will get the population and Foursquareco API to collect competitors on the same commune.

### 2.2) Data tools.

I first make use of Special Wards of Tokyo page from Wiki to scrap the table to create a data-frame. For this, I’ve used requests and Beautifulsoup4 library to create a data-frame containing name of the 23 wards of Tokyo, Area, population and 1st Major District.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
<
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0           conda-forge
    geopy:          

https://data.buenosaires.gob.ar/dataset/comunas


https://www.estadisticaciudad.gob.ar/eyc/wp-content/uploads/2018/05/MT_eah_2417.xlsx
    
https://www.estadisticaciudad.gob.ar/eyc/?p=82456

In [21]:
# Buenos Aires Open Data Catalogue - Neighbourhood Profiles 2016 (CSV)
# https://data.buenosaires.gob.ar/dataset/comunas

ipcf_df = pd.read_excel(r'https://www.estadisticaciudad.gob.ar/eyc/wp-content/uploads/2018/05/MT_eah_2417.xlsx', sheet_name='2019')
print('Data loaded')



Data loaded


In [22]:
ipcf_df.rename(columns={'Promedio del ingreso per cápita familiar (IPCF) de los hogares según comuna. Ciudad de Buenos Aires. Año 2019':'comunas'},inplace=True)
ipcf_df.rename(columns={'Unnamed: 1':'IPCF'},inplace=True)
ipcf_df.drop([0,1,17,18,19],axis=0,inplace=True)
ipcf_df.head(20)

,comunas,IPCF
2,1,35606
3,2,48847
4,3,28074
5,4,21631
6,5,33454
7,6,39050
8,7,27210
9,8,15187
10,9,23131
11,10,27730


A full report consisting of all of the following components (15 marks):
Introduction where you discuss the business problem and who would be interested in this project.
Data where you describe the data that will be used to solve the problem and the source of the data.
Methodology section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, if any, and what machine learnings were used and why.
Results section where you discuss the results.
Discussion section where you discuss any observations you noted and any recommendations you can make based on the results.
Conclusion section where you conclude the report.

In [23]:
comunas_df = pd.read_csv(r'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/comunas/comunas.csv')
print('Data loaded')

comunas_df.head(10)

Data loaded


,wkt,barrios,perimetro,area,comunas,id,objeto
0,MULTIPOLYGON (((-58.3800038855693 -34.57002352...,RECOLETA,21452.838648,6.317265e+06,2,1,LIMITE COMUNAL
1,MULTIPOLYGON (((-58.4128700313089 -34.61411625...,ALMAGRO - BOEDO,12323.432479,6.660603e+06,5,2,LIMITE COMUNAL
2,MULTIPOLYGON (((-58.4306058283309 -34.60704708...,CABALLITO,10990.964471,6.851029e+06,6,3,LIMITE COMUNAL
3,MULTIPOLYGON (((-58.4519952454565 -34.62974544...,FLORES - PARQUE CHACABUCO,17972.257870,1.242290e+07,7,4,LIMITE COMUNAL
4,MULTIPOLYGON (((-58.5192540532325 -34.63301009...,LINIERS - MATADEROS - PARQUE AVELLANEDA,21411.738344,1.650531e+07,9,5,LIMITE COMUNAL
5,MULTIPOLYGON (((-58.4883373673246 -34.62016035...,FLORESTA - MONTE CASTRO - VELEZ SARSFIELD - VE...,18332.037457,1.265656e+07,10,6,LIMITE COMUNAL
6,MULTIPOLYGON (((-58.4983848398831 -34.59636765...,VILLA DEL PARQUE - VILLA DEVOTO - VILLA GRAL....,19987.696155,1.412004e+07,11,7,LIMITE COMUNAL
7,MULTIPOLYGON (((-58.5033113250844 -34.59385615...,COGHLAN - SAAVEDRA - VILLA PUEYRREDON - VILLA ...,17232.189372,1.557093e+07,12,8,LIMITE COMUNAL
8,MULTIPOLYGON (((-58.4267593172764 -34.55202337...,PALERMO,22126.531858,1.584587e+07,14,9,LIMITE COMUNAL
9,MULTIPOLYGON (((-58.4119188098038 -34.59800307...,BALVANERA - SAN CRISTOBAL,10486.260809,6.385991e+06,3,10,LIMITE COMUNAL


In [24]:
comunas_df.drop('perimetro',axis=1,inplace=True)
comunas_df.drop('wkt',axis=1,inplace=True)
comunas_df.drop('objeto',axis=1,inplace=True)
comunas_df.drop('area',axis=1,inplace=True)
comunas_df.head()

,barrios,comunas,id
0,RECOLETA,2,1
1,ALMAGRO - BOEDO,5,2
2,CABALLITO,6,3
3,FLORES - PARQUE CHACABUCO,7,4
4,LINIERS - MATADEROS - PARQUE AVELLANEDA,9,5


In [25]:
buenosaires_merged = pd.merge(comunas_df, ipcf_df, on='comunas')
buenosaires_merged.drop('id',axis=1,inplace=True)
buenosaires_merged.head()

,barrios,comunas,IPCF
0,RECOLETA,2,48847
1,ALMAGRO - BOEDO,5,33454
2,CABALLITO,6,39050
3,FLORES - PARQUE CHACABUCO,7,27210
4,LINIERS - MATADEROS - PARQUE AVELLANEDA,9,23131


In [26]:
buenosaires_merged.rename(columns={'barrios':'neighboords','comunas':'communes','IPCF':'family per capita income'},inplace=True)
buenosaires_merged.head()

,neighboords,communes,family per capita income
0,RECOLETA,2,48847
1,ALMAGRO - BOEDO,5,33454
2,CABALLITO,6,39050
3,FLORES - PARQUE CHACABUCO,7,27210
4,LINIERS - MATADEROS - PARQUE AVELLANEDA,9,23131


In [33]:
population_data = pd.read_excel(r'https://www.estadisticaciudad.gob.ar/eyc/wp-content/uploads/2025/05/PBP_CO_1020.xls')
print('Data loaded')



Data loaded


In [55]:
population_df = pd.DataFrame(population_data, columns= ['Unnamed: 10','Proyecciones de población por comuna y sexo. Ciudad de Buenos Aires. Años 2010/2025'])
population_df.rename(columns={'Proyecciones de población por comuna y sexo. Ciudad de Buenos Aires. Años 2010/2025':'communes','Unnamed: 10':'2019'},inplace=True)
population_df.drop({0,1,2},axis=0,inplace=True)


In [60]:
population_df.drop({19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57},axis=0,inplace=True)
population_df

,2019,communes
3,255457.0,1
4,149510.0,2
5,193115.0,3
6,239712.0,4
7,187348.0,5
8,185271.0,6
9,241484.0,7
10,228266.0,8
11,171062.0,9
12,170497.0,10


In [78]:
buenosaires_merged2 = pd.merge(buenosaires_merged, population_df, on='communes')
buenosaires_merged2.rename(columns={'2019':'population'},inplace=True)
buenosaires_merged2.sort_values(by='communes', ascending=True,inplace=True)
buenosaires_merged2 = buenosaires_merged2.reset_index()
buenosaires_merged2.drop('index',axis=1,inplace=True)
buenosaires_merged2

,neighboords,communes,family per capita income,population
0,CONSTITUCION - MONSERRAT - PUERTO MADERO - RE...,1,35606,255457.0
1,RECOLETA,2,48847,149510.0
2,BALVANERA - SAN CRISTOBAL,3,28074,193115.0
3,BARRACAS - BOCA - NUEVA POMPEYA - PARQUE PATRI...,4,21631,239712.0
4,ALMAGRO - BOEDO,5,33454,187348.0
5,CABALLITO,6,39050,185271.0
6,FLORES - PARQUE CHACABUCO,7,27210,241484.0
7,VILLA LUGANO - VILLA RIACHUELO - VILLA SOLDATI,8,15187,228266.0
8,LINIERS - MATADEROS - PARQUE AVELLANEDA,9,23131,171062.0
9,FLORESTA - MONTE CASTRO - VELEZ SARSFIELD - VE...,10,27730,170497.0


In [27]:
!wget --quiet http://cdn.buenosaires.gob.ar/datosabiertos/datasets/comunas/CABA_comunas.geojson -O buenosaires.json
    
print('GeoJSON file downloaded!')

GeoJSON file downloaded!


In [28]:
world_geo = r'buenosaires.json' # geojson file

# create a plain world map
world_map = folium.Map(location=[0, 0], zoom_start=2, tiles='Mapbox Bright')


In [79]:
world_map.choropleth(
    geo_data=world_geo,
    data=buenosaires_merged,
    columns=['communes','family per capita income','population'],
    key_on='feature.properties.name',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='family per capita income'
)

# display map
world_map